In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [78]:
df_train = pd.read_csv('X_train.csv', encoding='cp949')
df_test = pd.read_csv('X_test.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv').gender
IDtest = df_test.custid.unique()

df_train.head()

,custid,sales_date,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,part_nm,team_nm,buyer_nm,import_flg,tot_amt,dis_amt,net_amt,inst_mon,inst_fee
0,0,2000-06-25 00:00:00,1212,무역점,2116050008000,에스티로더,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,90000,9000,81000,3,0
1,0,2000-06-25 00:00:00,1242,무역점,4125440008000,시슬리,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,39000,3900,35100,1,0
2,0,2000-08-26 00:00:00,1810,본점,2116052008000,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,1,175000,17500,157500,3,0
3,0,2000-08-26 00:00:00,1830,본점,4106430119900,듀퐁,수입의류,명품토탈,잡화파트,잡화가용팀,수입명품,1,455000,45500,409500,3,0
4,0,2000-09-03 00:00:00,1802,무역점,2139141008000,랑콤,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,0,100000,10000,90000,3,0


In [79]:
df_all = pd.concat([df_train, df_test])

In [80]:
df_all['sdate'] = df_all.sales_date.str[:10]
df_all["time"] = df_all['sdate'].astype(str).str.cat(df_all["sales_time"].astype(str), sep =" ")
df_all["time"]

def g(x):
    if len(str(x))==4:
        return(str(x)[0:2])
    else:
        return(str(x)[0:1])

df_all['sales_hour'] = df_all['sales_time'].apply(g)
df_all['sales_hour'] = df_all['sales_time'].apply(g)

In [81]:
df=df_all.groupby('goodcd')['tot_amt'].size().reset_index()
good=df.query('tot_amt>=30')['goodcd']

In [82]:
goodcd=df_all.query('goodcd in @good')

In [109]:
features=[]

In [110]:
f=pd.pivot_table(index='custid',columns='goodcd',values='tot_amt',aggfunc=np.size, fill_value=0,data=goodcd).reset_index()
features.append(f)

# 0.56대 변수

In [111]:
#평균 구매시간
df_all['hour']=df_all['sales_time']//100

f = df_all.groupby('custid')['hour'].agg([('평균구매시간', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f)

In [112]:
#충동구매 건수
f1 = df_all.groupby('custid')['hour'].agg([('평균구매시간', 'mean')]).reset_index()
f1.iloc[:,1] = f1.iloc[:,1].apply(round, args=(0,))
f2=df_all.groupby(['custid','hour'])['tot_amt'].agg([('구매건수','size')]).reset_index()
f3=pd.merge(f1,f2,on='custid')
f3['rate']=f3['평균구매시간']/f3['hour']

m1=f3.query('0<=rate<=1').groupby('custid')['구매건수'].agg([('평균 구매시간 내 구매건수','size')]).reset_index()
m2=f3.query('1<=rate').groupby('custid')['구매건수'].agg([('평균 구매시간 외 구매건수','size')]).reset_index()
m=pd.merge(m1,m2,on='custid')
m['평균 시간 구매비율']=m['평균 구매시간 내 구매건수']/m['평균 구매시간 외 구매건수']
m['충동구매 여부']=np.where(m['평균 시간 구매비율']<=1,'no','yes')
f = pd.get_dummies(m, columns=['충동구매 여부']) 
f=f3.query('1<=rate').groupby('custid')['구매건수'].agg([('충동구매 건수','sum')]).reset_index()
features.append(f)

In [113]:
#여성용,남성용 제품 중 남성용품 구매비율
a=pd.pivot_table(df_all, index='custid', columns='part_nm', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
women_brand = ['여성캐주얼','여성의류파트','여성정장','여성캐쥬얼']
man_bran = ['남성정장스포츠','남성의류']
f['남성용품 구매 비율']=a.loc[:,man_bran].sum(axis=1)/len(women_brand+man_bran)
f=f.iloc[:,[0,2]]
features.append(f)

In [114]:
#계절별 구매건수
def f1(x):
    k = x.month
    if 3 <= k <= 5 :
        return('봄-구매건수')
    elif 6 <= k <= 8 :
        return('여름-구매건수')
    elif 9 <= k <= 11 :    
        return('가을-구매건수')
    else :
        return('겨울-구매건수')    
    
df_all['season'] = pd.to_datetime(df_all.sales_date).apply(f1)
f = pd.pivot_table(df_all, index='custid', columns='season', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f)

In [115]:
# 아점저 시간대별 구매건수
def f2(x):
    if 9 <= x <= 12 :
        return('아침_구매건수')
    elif 13 <= x <= 17 :
        return('점심_구매건수')
    else :
        return('저녁_구매건수')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리

df_all['timeslot'] = (df_all.sales_time // 100).apply(f2)
f = pd.pivot_table(df_all, index='custid', columns='timeslot', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f)

In [116]:
# 새로운 시간대별 구매건수
def f2(x):
    if 901 <= x < 1200 :
        return('12시 이전_구매건수')
    elif 1200 <= x < 1400 :
        return('12~2시_구매건수')
    elif 1400 <= x < 1600 :
        return('2~4시_구매건수')
    elif 1600 <= x < 1800 :
        return('4~6시_구매건수')
    else :
        return('6시이후_구매건수')  

df_all['timeslot2'] = df_all.sales_time.apply(f2)

f = pd.pivot_table(df_all, index='custid', columns='timeslot2', values='tot_amt',
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f)

In [117]:
# 새로운 요일별 구매건수
def f2(x):
    k = x.dayofweek
    if k <= 2 :
        return('월화수_구매건수')
    elif 3 <= k < 5 :
        return('목금_구매건수')
    elif 5 <= k < 6 :
        return('토_구매건수')
    else :
        return('일_구매건수')    
    
df_all['요일2'] = pd.to_datetime(df_all.sales_date).apply(f2)
f = pd.pivot_table(df_all, index='custid', columns='요일2', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f)

In [118]:
# 시즌 변화 월기준의 구매건수
def f1(x):
    k = x.month
    if 2 <= k <= 4 :
        return('234월_구매건수')
    elif 5 <= k <= 7 :
        return('567월_구매건수')
    elif 8 <= k <= 10 :
        return('8910월_구매건수')
    else :
        return('11121월_구매건수')    
    
df_all['season2'] = pd.to_datetime(df_all.sales_date).apply(f1)
f = pd.pivot_table(df_all, index='custid', columns='season2', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f)

In [119]:
#실제 구매금액
f = df_all.groupby('custid')['net_amt'].agg([('실제구매금액', 'sum')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f)

In [120]:
# 총 구매액
f = df_all.groupby('custid')['tot_amt'].agg([('총구매액', 'sum')]).reset_index()
features.append(f)

In [121]:
# 구매 파트 변수의 각 파트의 빈도값 도출
f = pd.pivot_table(df_all, index='custid', columns='part_nm', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f)

# 구매제품 변수의 각 파트의 빈도값 도출
f = pd.pivot_table(df_all, index='custid', columns='buyer_nm', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f)

In [122]:
df = df_all.groupby(['custid','part_nm'])['tot_amt'].agg([('tot_amt_part', 'sum')]).reset_index()
df['part_nm'] = np.where(df.part_nm.str.contains('남성'), '남성', '여성')
df=df.pivot_table(values='tot_amt_part', index=df.custid, columns='part_nm', aggfunc='first',fill_value=0).reset_index()
df['남성파트구매비율'] = (df['남성'] / (df['남성'] + df['여성'])) * 100
df = df.fillna(0)
features.append(df.iloc[:,[0,-1]])

In [123]:
#지점별 구매총
df = df_all.groupby(['custid','str_nm'])['tot_amt'].agg([('tot_amt_str', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_str', index=df.custid, columns='str_nm', aggfunc='first', fill_value=0).reset_index()
features.append(df)

In [124]:
#커널변수
f = df_all.groupby('custid').agg({
    'tot_amt': [('최대구매액', 'max')],
    'dis_amt': [('dis_sum', 'sum'),('dis_mean', 'mean')],
    'inst_mon': [('최대할부개월수', 'max')],
    'brd_nm': [('구매상품다양성비', lambda x: x.nunique()/x.count())],
    'import_flg': [('수입상품_구매수', 'sum')],
    'sales_date': [('내점비율',lambda x: x.str[:10].nunique()/x.count())],
    'sales_time': [('저녁시간구입비율', lambda x: np.count_nonzero(x.astype(np.int)[(x>1800)|(x<900)])/ x.count())],
    }).reset_index()
features.append(f)

In [125]:
#총 구매건수
f = df_all.groupby('custid')['tot_amt'].agg([('구매건수', 'size')]).reset_index()
features.append(f)

In [170]:
X_train_y = pd.DataFrame({'custid': df_train.custid.unique()})
for f in features :
    X_train_y = pd.merge(X_train_y, f, how='left',on='custid')
display(X_train_y.shape)

X_test_y = pd.DataFrame({'custid': df_test.custid.unique()})
for f in features :
    X_test_y = pd.merge(X_test_y, f, how='left',on='custid')
display(X_test_y.shape)

(30000, 5108)

(19995, 5108)

In [173]:
#주말 요일별 구매비율
X_train_y["토_비율"] = (X_train_y["토_구매건수"] / X_train_y["구매건수"])*100
X_train_y["일_비율"] = (X_train_y["일_구매건수"] / X_train_y["구매건수"])*100
X_test_y["토_비율"] = (X_test_y["토_구매건수"] / X_test_y["구매건수"])*100
X_test_y["일_비율"] = (X_test_y["일_구매건수"] / X_test_y["구매건수"])*100

In [133]:
X_train_y.shape

(30000, 5110)

from sklearn.model_selection import train_test_split
y_train = pd.read_csv('y_train.csv').gender

X_train, X_test, y_train, y_test = train_test_split(
    X_train_y, y_train, random_state=0)

# Learn XGB
from xgboost import XGBClassifier
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

model = XGBClassifier(random_state=0, n_jobs=-1)
model.fit(X_train, y_train)

from sklearn.metrics import roc_auc_score
pred = model.predict_proba(X_test)[:,1]
pred
print(roc_auc_score(y_test, pred))

In [174]:
# 동일 구입파트 합쳐줌
X_train_y['가정용품R']=X_train_y['가정용품']+X_train_y['가정용품파트']
X_train_y['공산품R']=X_train_y['공산품']+X_train_y['공산품파트']
X_train_y['로얄부띠끄R']=X_train_y['로얄부띠끄']+X_train_y['로얄부틱']
X_train_y['생식품R']=X_train_y['생식품']+X_train_y['생식품파트']
X_train_y['스포츠캐주얼R']=X_train_y['스포츠캐주얼']+X_train_y['스포츠캐쥬얼']
X_train_y['여성캐주얼R']=X_train_y['여성캐주얼']+X_train_y['여성캐쥬얼']
X_train_y['잡화R']=X_train_y['잡화']+X_train_y['잡화파트']

X_test_y['가정용품R']=X_test_y['가정용품']+X_test_y['가정용품파트']
X_test_y['공산품R']=X_test_y['공산품']+X_test_y['공산품파트']
X_test_y['로얄부띠끄R']=X_test_y['로얄부띠끄']+X_test_y['로얄부틱']
X_test_y['생식품R']=X_test_y['생식품']+X_test_y['생식품파트']
X_test_y['스포츠캐주얼R']=X_test_y['스포츠캐주얼']+X_test_y['스포츠캐쥬얼']
X_test_y['여성캐주얼R']=X_test_y['여성캐주얼']+X_test_y['여성캐쥬얼']
X_test_y['잡화R']=X_test_y['잡화']+X_test_y['잡화파트']

In [175]:
# 각성별 구입건수가 많은 파트 묶어주기
X_train_y['남성파트']=X_train_y['가정용품R']+X_train_y['공산품R']+X_train_y['생식품R']+X_train_y['케주얼,구두,아동']
X_test_y['남성파트']=X_test_y['가정용품R']+X_test_y['공산품R']+X_test_y['생식품R']+X_test_y['케주얼,구두,아동']
X_train_y['여성파트']=X_train_y['여성캐주얼']+X_train_y['영캐릭터']+X_train_y['영플라자']+X_train_y['패션잡화'] 
X_test_y['여성파트']=X_test_y['여성캐주얼']+X_test_y['영캐릭터']+X_test_y['영플라자']+X_test_y['패션잡화'] 

In [176]:
# 1206 새로운 변수 리코드하고 추가하기
def f1(x):
    if  x <= 2000000 :
        return('1')
    elif 2000001 <= x < 5000000 :
        return('2')
    else :
        return('3')  

X_train_y['총구매액B'] = X_train_y.총구매액.apply(f1)
X_train_y['총구매액B'] = X_train_y['총구매액B'].astype(int)

In [177]:
def f1(x):
    if  x <= 2000000 :
        return('1')
    elif 2000001 <= x < 5000000 :
        return('2')
    else :
        return('3')  

X_test_y['총구매액B'] = X_test_y.총구매액.apply(f1)
X_test_y['총구매액B'] = X_test_y['총구매액B'].astype(int)

In [178]:
def f1(x):
    if  x <= 1500 :
        return('4')
    elif 1501 <= x < 1600 :
        return('3')
    elif 1601 <= x < 1700 :
        return('2')
    else :
        return('1')  

X_train_y['평균구매시간B'] = X_train_y.평균구매시간.apply(f1)
X_train_y['평균구매시간B'] = X_train_y['평균구매시간B'].astype(int)

In [179]:
def f1(x):
    if  x <= 1500 :
        return('4')
    elif 1501 <= x < 1600 :
        return('3')
    elif 1601 <= x < 1700 :
        return('2')
    else :
        return('1')  

X_test_y['평균구매시간B'] = X_test_y.평균구매시간.apply(f1)
X_test_y['평균구매시간B'] = X_test_y['평균구매시간B'].astype(int)

In [180]:
def f1(x):
    if  x < 1 :
        return('1')
    elif 1 <= x <= 10 :
        return('2')
    elif 11 <= x <= 20 :
        return('3')
    else :
        return('4')  

X_train_y['남성파트B'] = X_train_y.남성파트.apply(f1)
X_train_y['남성파트B'] = X_train_y['남성파트B'].astype(int)

In [181]:
def f1(x):
    if  x < 1 :
        return('1')
    elif 1 <= x <= 10 :
        return('2')
    elif 11 <= x <= 20 :
        return('3')
    else :
        return('4')  

X_test_y['남성파트B'] = X_test_y.남성파트.apply(f1)
X_test_y['남성파트B'] = X_test_y['남성파트B'].astype(int)

In [182]:
def f1(x):
    if  x < 1 :
        return('3')
    elif 1 <= x <= 4 :
        return('2')
    else :
        return('1')  

X_train_y['여성파트B'] = X_train_y.여성파트.apply(f1)
X_train_y['여성파트B'] = X_train_y['여성파트B'].astype(int)

In [183]:
def f1(x):
    if  x < 1 :
        return('3')
    elif 1 <= x <= 4 :
        return('2')
    else :
        return('1')  

X_test_y['여성파트B'] = X_test_y.여성파트.apply(f1)
X_test_y['여성파트B'] = X_test_y['여성파트B'].astype(int)

In [184]:
X_train_y['구매_시간'] = X_train_y['총구매액B'] * X_train_y['평균구매시간B']
X_train_y['남성_여성'] = X_train_y['남성파트B'] * X_train_y['여성파트B']
X_train_y['구매_남성'] = X_train_y['총구매액B'] * X_train_y['남성파트B']
X_train_y['구매_여성'] = X_train_y['총구매액B'] * X_train_y['여성파트B']
X_train_y['시간_남성'] = X_train_y['평균구매시간B'] * X_train_y['남성파트B']
X_train_y['시간_여성'] = X_train_y['평균구매시간B'] * X_train_y['여성파트B']

X_test_y['구매_시간'] = X_test_y['총구매액B'] * X_test_y['평균구매시간B']
X_test_y['남성_여성'] = X_test_y['남성파트B'] * X_test_y['여성파트B']
X_test_y['구매_남성'] = X_test_y['총구매액B'] * X_test_y['남성파트B']
X_test_y['구매_여성'] = X_test_y['총구매액B'] * X_test_y['여성파트B']
X_test_y['시간_남성'] = X_test_y['평균구매시간B'] * X_test_y['남성파트B']
X_test_y['시간_여성'] = X_test_y['평균구매시간B'] * X_test_y['여성파트B']

In [171]:
# 치우친 데이터 로그 치환하기
f = X_train_y.총구매액.where(X_train_y.총구매액>=0, other=0)
f = np.log(f+1)
X_train_y.총구매액 = f

f = X_test_y.총구매액.where(X_test_y.총구매액>=0, other=0)
f = np.log(f+1)
X_test_y.총구매액 = f

#f = X_train_y.평균구매가격.where(X_train_y.평균구매가격>=0, other=0)
#f = np.log(f+1)
#X_train_y.평균구매가격 = f

#f = X_test_y.평균구매가격.where(X_test_y.평균구매가격>=0, other=0)
#f = np.log(f+1)
#X_test_y.평균구매가격 = f

f = X_train_y.구매건수.where(X_train_y.구매건수>=0, other=0)
f = np.log(f+1)
X_train_y.구매건수 = f

f = X_test_y.구매건수.where(X_test_y.구매건수>=0, other=0)
f = np.log(f+1)
X_test_y.구매건수 = f

#f = X_train_y.내점일수.where(X_train_y.내점일수>=0, other=0)
#f = np.log(f+1)
#X_train_y.내점일수 = f

#f = X_test_y.내점일수.where(X_test_y.내점일수>=0, other=0)
#f = np.log(f+1)
#X_test_y.내점일수 = f


#f = X_train_y.실제구매금액평균.where(X_train_y.실제구매금액평균>=0, other=0)
#f = np.log(f+1)
#X_train_y.실제구매금액평균 = f

#f = X_test_y.실제구매금액평균.where(X_test_y.실제구매금액평균>=0, other=0)
#f = np.log(f+1)
#X_test_y.실제구매금액평균 = f

f = X_train_y.실제구매금액.where(X_train_y.실제구매금액>=0, other=0)
f = np.log(f+1)
X_train_y.실제구매금액 = f

f = X_test_y.실제구매금액.where(X_test_y.실제구매금액>=0, other=0)
f = np.log(f+1)
X_test_y.실제구매금액 = f

In [185]:
IDtest = X_test_y.custid;
X_train_y.drop(['custid'], axis=1, inplace=True)
X_test_y.drop(['custid'], axis=1, inplace=True)

In [186]:
print(X_train_y.shape)
print(X_test_y.shape)

(30000, 5128)
(19995, 5128)


# 피벗 나열

In [187]:
X_train12 = pd.pivot_table(df_all, index='custid', values='net_amt',
                         aggfunc=np.sum , fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values

X_test12 = pd.pivot_table(df_all, index='custid', values='net_amt',
                         aggfunc=np.sum , fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values

X_train12 = pd.DataFrame(X_train12)
X_test12 = pd.DataFrame(X_test12)

In [188]:
X_train8 = pd.pivot_table(df_all, index='custid', values='tot_amt',
                         aggfunc=np.sum , fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values

X_test8 = pd.pivot_table(df_all, index='custid', values='tot_amt',
                         aggfunc=np.sum , fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values

X_train8 = pd.DataFrame(X_train8)
X_test8 = pd.DataFrame(X_test8)

In [189]:
level = 'team_nm'
df_train[level].nunique()

X_train6 = pd.pivot_table(df_all, index='custid', columns=level, values='tot_amt',
                         aggfunc=np.size , fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values

X_test6 = pd.pivot_table(df_all, index='custid', columns=level, values='tot_amt',
                         aggfunc=np.size , fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values

X_train6 = pd.DataFrame(X_train6)
X_test6 = pd.DataFrame(X_test6)

In [190]:
level = 'pc_nm'
df_train[level].nunique()

X_train4 = pd.pivot_table(df_all, index='custid', columns=level, values='tot_amt',
                         aggfunc= np.size , fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values

X_test4 = pd.pivot_table(df_all, index='custid', columns=level, values='tot_amt',
                         aggfunc= np.size , fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values

X_train4 = pd.DataFrame(X_train4)
X_test4 = pd.DataFrame(X_test4)

In [191]:
level = 'corner_nm'
df_train[level].nunique()

X_train3 = pd.pivot_table(df_all, index='custid', columns=level, values='tot_amt',
                         aggfunc= np.size , fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values

X_test3 = pd.pivot_table(df_all, index='custid', columns=level, values='tot_amt',
                         aggfunc= np.size , fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values

X_train3 = pd.DataFrame(X_train3)
X_test3 = pd.DataFrame(X_test3)

In [192]:
level = 'brd_nm'
df_train[level].nunique()

X_train2 = pd.pivot_table(df_all, index='custid', columns=level, values='tot_amt',
                         aggfunc= np.size , fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values

X_test2 = pd.pivot_table(df_all, index='custid', columns=level, values='tot_amt',
                         aggfunc= np.size , fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values

X_train2 = pd.DataFrame(X_train2)
X_test2 = pd.DataFrame(X_test2)

In [196]:
df_all['sdate'] = df_all.sales_date.str[:10]
df_all["time"] = df_all['sdate'].astype(str).str.cat(df_all["sales_time"].astype(str), sep =" ")
df_all["time"]

def g(x):
    if len(str(x))==4:
        return(str(x)[0:2])
    else:
        return(str(x)[0:1])

df_train['sales_hour'] = df_train['sales_time'].apply(g)
df_train['sales_hour'] = df_train['sales_time'].apply(g)

In [ ]:
level = 'sales_hour'
df_train[level].nunique()
X_train1 = pd.pivot_table(df_all, index='custid', columns=level,values='tot_amt',
                         aggfunc=np.size , fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values

X_test1 = pd.pivot_table(df_all, index='custid', columns=level,values='tot_amt',
                         aggfunc=np.size , fill_value=0). \
                         reset_index(). \
                         query('custid  in @IDtest'). \
                         drop(columns=['custid']).values

X_train1 = pd.DataFrame(X_train1)
X_test1 = pd.DataFrame(X_test1)

In [ ]:
X_train = pd.concat([X_train1,X_train2],axis=1)
X_train = pd.concat([X_train,X_train3],axis=1)
X_train = pd.concat([X_train,X_train4],axis=1)
#X_train = pd.concat([X_train,X_train5],axis=1)
X_train = pd.concat([X_train,X_train6],axis=1)
#X_train = pd.concat([X_train,X_train7],axis=1)
X_train = pd.concat([X_train,X_train8],axis=1)
#X_train = pd.concat([X_train,X_train9],axis=1)
#X_train = pd.concat([X_train,X_train10],axis=1)
#X_train = pd.concat([X_train,X_train11],axis=1)
X_train = pd.concat([X_train,X_train12],axis=1)
#X_train = pd.concat([X_train,X_train13],axis=1)
#X_train = pd.concat([X_train,X_train14],axis=1)
#X_train = pd.concat([X_train,X_train15],axis=1)


X_test = pd.concat([X_test1,X_test2],axis=1)
X_test = pd.concat([X_test,X_test3],axis=1)
X_test = pd.concat([X_test,X_test4],axis=1)
#X_test = pd.concat([X_test,X_test5],axis=1)
X_test = pd.concat([X_test,X_test6],axis=1)
#X_test = pd.concat([X_test,X_test7],axis=1)
X_test = pd.concat([X_test,X_test8],axis=1)
#X_test = pd.concat([X_test,X_test9],axis=1)
#X_test = pd.concat([X_test,X_test10],axis=1)
#X_test = pd.concat([X_test,X_test11],axis=1)
X_test = pd.concat([X_test,X_test12],axis=1)
#X_test = pd.concat([X_test,X_test13],axis=1)
#X_test = pd.concat([X_test,X_test14],axis=1)
#X_test = pd.concat([X_test,X_test15],axis=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train_all=pd.concat([X_train,X_train_y],axis=1) 
X_test_all=pd.concat([X_test,X_test_y],axis=1)

print(X_train_all.shape) 
print(X_test_all.shape)